In [1]:
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline
from cmapPy.pandasGEXpress.parse import parse
from scipy.stats import spearmanr as scor
from scipy.stats import mannwhitneyu as mwu

Importing the Niche Net matrix

In [3]:
nn_model=pd.read_csv('../data/ligand_target_matrix.csv', sep=',', header=0, index_col=0)

In [4]:
nn_model.head()

,CXCL1,CXCL2,CXCL3,CXCL5,PPBP,CXCL6,CXCL8,CXCL9,CXCL10,CXCL11,...,PCDHGB1,CDH6,CDH9,CDH12,CDH18,PCDHB16,PCDHA7,PCDHA6,PCDHA4,PCDHA12
A1BG,0.000353,0.000404,0.000373,0.000308,0.000263,0.000273,0.000414,0.000298,0.000416,0.000308,...,0.000303,0.000319,0.000323,0.000348,0.000393,0.000334,0.000335,0.000288,0.000366,0.000323
A1BG-AS1,0.000165,0.000151,0.000158,0.000132,0.000123,0.000118,0.000217,0.000141,0.000170,0.000137,...,0.000160,0.000164,0.000164,0.000178,0.000211,0.000145,0.000166,0.000140,0.000184,0.000175
A1CF,0.000579,0.000460,0.000390,0.000329,0.000321,0.000309,0.000483,0.000370,0.000393,0.000358,...,0.000425,0.001664,0.001589,0.000940,0.001422,0.000432,0.000575,0.000536,0.000517,0.000403
A2M,0.000603,0.000600,0.000516,0.000452,0.000459,0.000416,0.000703,0.000541,0.000616,0.000497,...,0.000473,0.000508,0.000497,0.000587,0.000570,0.000514,0.000536,0.000497,0.000572,0.000554
A2M-AS1,0.000089,0.000082,0.000075,0.000049,0.000051,0.000048,0.000138,0.000055,0.000061,0.000080,...,0.000032,0.000078,0.000077,0.000079,0.000080,0.000033,0.000081,0.000005,0.000050,0.000080


Importing single perturbed genes from CREEDs

In [9]:
gene_perturbed=pd.read_csv('../data/single_gene_perturbations-p1.0.csv', sep=',', header=0, index_col=0)

In [10]:
gene_perturbed.tail()

,cell_type,ctrl_ids,geo_id,hs_gene_symbol,organism,pert_ids,platform,version
id,,,,,,,,
gene:P9634,T-Lymphocyte|Cultured Cell Line,GSM760729|GSM760730|GSM760731,GSE30674,T-lymphocyte activation antigen CD80|Rho GTPas...,human,GSM760754|GSM760755,GPL570,2.0
gene:P11479,Entire liver|Blood - brain barrier anatomy|Bra...,GSM877947|GSM877948,GSE35943,Signal transducer and activator of transcripti...,mouse,GSM877875|GSM877876,GPL7202,2.0
gene:P11486,Entire liver|Blood - brain barrier anatomy|Bra...,GSM877947|GSM877948,GSE35943,Signal transducer and activator of transcripti...,mouse,GSM877920|GSM877921,GPL7202,2.0
gene:P11487,Entire liver|Blood - brain barrier anatomy|Bra...,GSM877947|GSM877948,GSE35943,Signal transducer and activator of transcripti...,mouse,GSM877922|GSM877923,GPL7202,2.0
gene:P11484,Entire liver|Blood - brain barrier anatomy|Bra...,GSM877947|GSM877948,GSE35943,Signal transducer and activator of transcripti...,mouse,GSM877916|GSM877917,GPL7202,2.0


In [17]:
gene_perturbed.shape

(8620, 8)